# Моделирование нагрузки велокурьеров (на примере склада №5018)

Велозона разбита на несколько сегментов: Кушелевская, Лесная, Блюхер, Кондратьевский-верх, Кондратьевский-низ, Революция (часовая зона)

Формальная очередь одна для всех зон

Неформальных очередей 6:
Кушелевская+Блюхер(опционально),
Лесная+Блюхер(опционально),
Кондратьевский-верх+Блюхер(опционально),
Кондратьевский-низ зона 30 мин,
Кондратьевский-низ часовая зона + Революция (часовая зона),
Экспрессы (все зоны)

Курьеры распределены по четырем группам:
Группа "Экспресс" (2-5 чел)
Группа "Ревал" (1-3 чел)
Группа "30-минут" (2-4 чел)
Группа "Общая" (5-25 чел)

Координация между группами отсутствует, в случае "подгорания" заказов, куртор или тот кто его замещает, говорит (т.е. это ручная работа) кому и куда ехать (в этом случае курьер берется из любой группы, как правило из общей).

Курьер смотрит через мобильное приложение "ВкусВилл курьер" на формальную очередь, в которой отображаются все заказы.
Одновременно он смотрит на список каналов в телеграм, где отображаются "неформальные" очереди. Курьер встает в очередь, ставя плюс в неформальную очередь.
После завершения набора, он ставит себе 👍 (лайк)

Группа "Общая" обслуживает одновременно три "неформальных" очереди: Кушелевская+Блюхер(опционально), Лесная+Блюхер(опционально), Кондратьевский-верх+Блюхер(опционально)
Группа "Экспресс" обслуживает одну "неформальную" очередь: Экспрессы
Группа "Ревал" обслуживает одну "неформальную" очередь: Кондратьевский-низ часовая зона + Революция (часовая зона)
Группа "30-минут" обслуживает одну "неформальную" очередь: Кондратьевский-низ зона 30 мин

Некоторое время назад была введена "блокировка" формальной очереди, речи идет о галочке, когда курьер должен взять первый заказ из формальной очереди для начала формирования маршрута. Ниже, исследование влияния этой блокировки на общую пропускную способность потока заказов.

Файл с нагрузкой, сгенерирован путем равномерного размазывания заказов на неделю. Используются рандомные значения, за неимением доступа к реальным данным.

Модель поведения курьера: Курьер набирается по максимуму что бы отвезти как можно больше заказов и не сделать "просрок", подобная модель поведения приводит к прижатию к верхней границе SLA для заказа.


In [3]:
from os import environ
from os.path import exists

import pandas as pd
from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

orders_generator = pd.read_csv('./order-addresses.csv')

# LOAD GEOCODING-CACHE
YANDEX_JAVASCRIPT_API_KEY = environ.get('YANDEX_JAVASCRIPT_API_KEY', '')
if not YANDEX_JAVASCRIPT_API_KEY:
    raise Exception('Invalid yandex javascript API key')

# REFRESH-GEOCODING-CACHE
GEOCODING_CACHE_FILENAME = './.geocoding_cache.csv'
_geocoding_cache_csv = pd.read_csv(GEOCODING_CACHE_FILENAME) if exists(GEOCODING_CACHE_FILENAME) else pd.DataFrame(columns=['geocoding_cache_address','geocoding_cache_lat','geocoding_cache_lon'])
_geocoding_cache_csv.to_csv(GEOCODING_CACHE_FILENAME, mode='x')

# orders_generator.tail()

orders_generator['order_address'].tail()

15    1-й Муринский проспект 19
16         Лесной проспект 37к6
17        Новолитовская улица 4
18        Новолитовская улица 5
19         Лесной проспект 67к1
Name: order_address, dtype: object

## методика вычисления среднего времени доставки и риска "просрока"

Среднее время доставки, интегральный показатель, который вычисляется по всем доставленным заказам.

$D = \frac{\sum_{n=1}^{N} t_n}{N}$ где $N$ общее количество заказов за период, $t_n$ время доставки.

Риск просрока вычисляется исходя из близости времени доставки к SLA.

$R = \frac{\sum_{n=1}^{N} (T_n - t_n)}{N}$ где $N$ общее количество заказов за период, $T_n$ SLA зоны доставки, $t_n$ время доставки.